## ceo 이름 찾기

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys

In [3]:
firm_df = pd.read_excel('OC_KR.xlsx', engine='openpyxl')
firm_name_list = firm_df['Organization Name'].tolist()

In [8]:
url = 'http://www.google.com'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

In [ ]:
firm_list = []
CEO_infomation = []

#for i in range(len(firm_name_list[50:60])):
for i in firm_name_list:
    print(i)
    driver.get(url)
    
    firm_name = i +' 대표'
    
    # 검색
    search_box = driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
    search_box.clear()
    search_box.send_keys(firm_name)
    search_box.send_keys(u'\ue007')
#     try:
#         driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[2]/div[2]/div[2]/center/input[1]').click()
#     except:
#         driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').click()
    
    time.sleep(2)
    
    # CEO 정보 본문내용 우선 저장
    temp_list = []
    temp = driver.find_elements_by_css_selector('div > div > div.IsZvec > div')
    for j in temp:
        temp_list.append(j.text)
    CEO_text = temp_list
    
    try:
        # CEO 정보
        temp_text = driver.find_element_by_xpath('//*[@id="rso"]/div[1]/div/div[1]/div/div[1]/div').text
        if temp_text != '':
            CEO_text = temp_text
    except:
        print(i + ' error')
            

    # 회사명, CEO 정보 각 list에 추가
    firm_list.append(i)
    CEO_infomation.append(CEO_text)

In [ ]:
results = pd.DataFrame({'firm_name':firm_list,'CEO_information':CEO_infomation})

In [ ]:
results

## ceo 기사 (전체 기사 / confident & conservative 기사)
- API: 회사이름으로 기사 전체 갯수 출력  
- API: 회사이름+CEO 이름+confident(자신, 희망, 낙관, 긍정, 확신, 두려움없, 대담, 용기) OR conservative(보수, 비관, 믿음, 차분, 한결, 조심, 중도, 신중) 

## linked in - mkt 담당자 숫자 확인 

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import time
import bs4
pd.set_option('display.max_columns', None)
from selenium.webdriver.common.keys import Keys

In [3]:
firm_df = pd.read_excel('OC_KR.xlsx', engine='openpyxl')
firm_list = firm_df['Organization Name'][firm_df['LinkedIn'].notnull()].tolist()

In [4]:
Linkedin_list = firm_df['LinkedIn'][firm_df['LinkedIn'].notnull()].tolist()

In [18]:
import re

pattern = r'https://www.linkedin.com/company/(.*)/?'
expected_urls = []
unexpected_urls = []
for url in Linkedin_list:

    company_names = re.findall(pattern, url)
    if company_names: # 정상적인 URL
        company_name = company_names[0]
        expected_urls.append(
            f"https://www.linkedin.com/company/{company_name}/people"
        )

    else: # 비 정상적인 URL -> 무시
        unexpected_urls.append(url)
        continue

len(expected_urls), len(unexpected_urls)
Linkedin_list = expected_urls

In [6]:
# Linkedin URL cleaning
temp_list = []
for i in Linkedin_list:
    front_url = i.split('company/')[0]
    try:
        company_name = i.split('company/')[1].split('/')[0]
    except:
        company_name = ''
    temp_url = front_url + 'company/' + company_name + '/people'
    temp_list.append(temp_url)
    
Linkedin_list = temp_list
Linkedin_list

['https://www.linkedin.cn/company/spursell/people',
 'https://www.linkedin.com/companycompany//people',
 'https://www.linkedin.com/company/%EB%A3%A8%EB%82%98%EC%86%8C%ED%94%84%ED%8A%B8/people',
 'https://www.linkedin.com/company/%EC%A3%BC%EC%8B%9D%ED%9A%8C%EC%82%AC-%ED%8E%AB%EB%9D%BC%EC%9D%B4%ED%81%AC/people',
 'https://www.linkedin.com/company/%ED%95%80%EB%8B%A4-finda-/people',
 'https://www.linkedin.com/company/10099935?trk=tyah&trkInfo=clickedVertical%3Acompany%2CclickedEntityId%3A10099935%2Cidx%3A2-1-3%2CtarId%3A1459180716666%2Ctas%3An.code/people',
 'https://www.linkedin.com/company/1am/people',
 'https://www.linkedin.com/company/6gram/people',
 'https://www.linkedin.com/company/althea-inc-/people',
 'https://www.linkedin.com/company/b2link-corporation/people',
 'https://www.linkedin.com/company/bettercode-co.-ltd./people',
 'https://www.linkedin.com/company/breezm/people',
 'https://www.linkedin.com/company/brich-%EB%B8%8C%EB%A6%AC%EC%B9%98-/people',
 'https://www.linkedin.com/co

In [6]:
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(10)

In [7]:
url = 'https://www.linkedin.com/login'
driver.get(url)

In [8]:
# 기존 회원 로그인 버튼 클릭
# try:
#     origin_xpath = '/html/body/div[1]/main/p/a'
#     new_xpath = '/html/body/header/nav/div/a[2]'
#     driver.find_element_by_xpath(new_xpath).click()
# except:
#     print('Error')
#     None

# target_text = '로그인'
# elements = driver.find_elements_by_xpath(f"//*[contains(text(), '{target_text}')]")
# login_btn = list(filter(lambda e: e.text == target_text, elements))
# if login_btn:
#     login_btn = login_btn.pop()
# else:
#     raise RuntimeError(f"Can't find any {target_text} element")
# login_btn.click()

# ID 입력

elem_login = driver.find_element_by_xpath('//*[@id="username"]')
elem_login.clear()
elem_login.send_keys('charmbjh@korea.ac.kr')

# 비밀번호 입력
elem_login = driver.find_element_by_xpath('//*[@id="password"]')
elem_login.clear()
elem_login.send_keys('kate0987')

time.sleep(2)

# 로그인 버튼 클릭
driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button').click()

# 나중에 버튼 클릭
try :
    driver.find_element_by_xpath('//*[@id="remember-me-prompt__form-secondary"]/button').click()
except :
    print('None')

None


In [28]:
# f"//*[contains(text(), '{target_text}')]"
from time import sleep
driver.get('https://www.linkedin.com/company/%EC%A3%BC%EC%8B%9D%ED%9A%8C%EC%82%AC-%ED%8E%AB%EB%9D%BC%EC%9D%B4%ED%81%AC/people/')
target_text = '직원 0명'
sleep(4)
elements = driver.find_elements_by_xpath(f"//*[contains(text(), '{target_text}')]")
elements
# for e in elements:
#     re.findall(r"직원 ([0-9]*)명", e.text)
#     print(e, e.text)
#     print(re.findall(r"직원 ([0-9]*)명", e.text))

[]

In [31]:
firm_name = []
num_total_list = []
specific_list = []
num_mkt_list = []
mkt_proportion_list = [] 

for i in range(len(Linkedin_list)):
    
    driver.get(Linkedin_list[i])
    print(Linkedin_list[i])
    
    time.sleep(2)
    # 총 재직자 수
    try:
        num_total_employee = int(driver.find_element_by_xpath('//*[@id="main"]/div[2]/div/div[1]/div[1]').text.split(' ')[1].replace('명',''))
        print(f"총 재직자 수: {num_total_employee}")
    except:
        print("No 총 재직자 수")
        continue

    # 만약 재직자 수가 0명일 경우 Pass
    if not num_total_employee:
        firm_name.append(firm_list[i])
        num_total_list.append(num_total_employee)
        specific_list.append(None)
        num_mkt_list.append(None)
        mkt_proportion_list.append(None)
        continue

    # 다음
    driver.find_element_by_xpath('/html/body/div[5]/div[3]/div/div[2]/div/div[2]/main/div[2]/div/div[1]/div[2]/div/section/div[1]/div/div/button[2]').send_keys(Keys.ENTER)

    # show more
    driver.find_element_by_css_selector('#main > div.org-grid__content-height-enforcer > div > div.artdeco-card.pb2 > div.relative > button').send_keys(Keys.ENTER)

    # position
    position = driver.find_element_by_xpath('/html/body/div[5]/div[3]/div/div[2]/div/div[2]/main/div[2]/div/div[1]/div[2]/div/section/div[2]/ul/li[3]/div/div')
    num_job = position.text.split('\n')[1:]
    
    temp = pd.Series(num_job).apply(lambda x:x.split(' '))
    position_dict = dict(zip(temp.apply(lambda x:x[1]), temp.apply(lambda x:x[0])))
    
    try :
        num_mkt = int(position_dict['마케팅'])

    except :
        num_mkt = 0
    
    # 마케팅 인원 비율
    mkt_proportion = num_mkt / num_total_employee
    
    # 리스트 저장
    firm_name.append(firm_list[i])
    num_total_list.append(num_total_employee)
    specific_list.append(num_job)
    num_mkt_list.append(num_mkt)
    mkt_proportion_list.append(mkt_proportion)

https://www.linkedin.com/company/%EB%A3%A8%EB%82%98%EC%86%8C%ED%94%84%ED%8A%B8//people
총 재직자 수: 21
https://www.linkedin.com/company/%EC%A3%BC%EC%8B%9D%ED%9A%8C%EC%82%AC-%ED%8E%AB%EB%9D%BC%EC%9D%B4%ED%81%AC//people
총 재직자 수: 0


NameError: name 'irm_name' is not defined

In [53]:
results = pd.DataFrame({'firm_name':firm_name,
                       'num_total_employee':num_total_list,
                       'num_job':specific_list,
                       'num_mkt':num_mkt_list,
                       'mkt_proportion':mkt_proportion_list})

In [55]:
firm_name

[]

## 투자 금액 
- 회사이름 + 유치 

## 페이스북 팔로워 숫자 세기

In [ ]:
import os
import csv

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [ ]:
class TwitterCralwer:
    def __init__(self, driverpath=None):
        default_path = os.path.join(os.getcwd(), "chromedriver")
        if driverpath is None:
            driverpath = default_path

        options = webdriver.ChromeOptions()
        # options.add_argument('headless')

        self.driver = webdriver.Chrome(driverpath, options=options)

    def get(self, url, css_selector):
        self.driver.get(url)

        try:
            WebDriverWait(self.driver, 5).until(
                EC.presence_of_element_located((By.ID, css_selector))
            )
        except TimeoutException:
            pass

        html = self.driver.page_source

        soup = bs(html, features="html.parser")

        res = None
        try:
            res = soup.select(css_selector)[-1].text
        except IndexError:
            print("This URL may be not valid")

        return res


if __name__ == '__main__':
    css_selector = "span.css-901oao.css-16my406.r-1fmj7o5.r-poiln3.r-b88u0q.r-bcqeeo.r-qvutc0 > span"
    # #mount_0_0_r4 > div > div:nth-child(1) > div > div.rq0escxv.l9j0dhe7.du4w35lb > div > div > div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div.j83agx80.cbu4d94t.dp1hu0rb > div > div > div.bp9cbjyn.j83agx80.cbu4d94t.d2edcug0 > div.rq0escxv.d2edcug0.ecyo15nh.hv4rvrfc.dati1w0a.tr9rh885 > div > div.rq0escxv.l9j0dhe7.du4w35lb.qmfd67dx.hpfvmrgz.o387gat7.buofh1pr.g5gj957u.aov4n071.oi9244e8.bi6gxh9e.h676nmdw.aghb5jc5.rek2kq2y > div.lpgh02oy > div > div:nth-child(1) > div > div > div > div > div.sej5wr8e > div > div > ul > div.rq0escxv.l9j0dhe7.du4w35lb.j83agx80.pfnyh3mw.jifvfom9.gs1a9yip.owycx6da.btwxx1t3.jb3vyjys.b5q2rw42.lq239pai.mysgfdmx.hddg9phg > div.rq0escxv.l9j0dhe7.du4w35lb.j83agx80.cbu4d94t.g5gj957u.d2edcug0.hpfvmrgz.rj1gh0hx.buofh1pr.o8rfisnq.p8fzw8mz.pcp91wgn.iuny7tx3.ipjc6fyt > div > div > span
    tc = TwitterCralwer()

    with open("list.csv", "r") as file:
        reader = csv.reader(file)

        urls = [row[6] for row in reader if row[6]]
        urls = urls[1:]

    with open("result.csv", "w") as file:
        writer = csv.writer(file)
        for idx, url in enumerate(urls):
            print(idx, url)
            writer.writerow([url, tc.get(url, css_selector)])

    # url = "http://www.twitter.com/MotherlyMedia"
    # print(tc.get(url, css_selector))

    tc.driver.close()
